In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

df = yf.download("AAPL", start="2026-01-07", interval='1m')
df = df[["Open", "High", "Low", "Close", "Volume"]].dropna()
# df.head()

if isinstance(df.columns, pd.MultiIndex):
    df.columns = df.columns.get_level_values(0)
df.head()